In [1]:
Sys.setlocale(category = "LC_ALL", locale = "en_US.UTF-8")

[1] "en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/C"

# Dependencies

# Functions

In [2]:
# compute_divstats_parallel <- function(lst, chain, batchname, outpath, cores = max(1, parallel::detectCores() - 1)) {
#     if (!requireNamespace("iNEXT", quietly = TRUE)) stop("Package 'iNEXT' is required.")
#     if (!requireNamespace("parallel", quietly = TRUE)) stop("Package 'parallel' is required.")

#     sample_names <- names(lst)
#     if (is.null(sample_names) || any(!nzchar(sample_names))) {
#         sample_names <- paste0("sample_", seq_along(lst))
#     }

#     iNEXT_list <- parallel::mclapply(
#         X = lst,
#         FUN = function(x) iNEXT::iNEXT(x, q = 0, datatype = "abundance"),
#         mc.cores = cores
#     )
#     names(iNEXT_list) <- sample_names

#     extract_qD_max <- function(res) {
#         est <- res$iNextEst
#         if (is.list(est)) est <- est[[1]]
#         if (!is.data.frame(est) || !"qD" %in% names(est)) return(NA_real_)
#         max(est$qD, na.rm = TRUE)
#     }
#     extract_SC_max <- function(res) {
#         est <- res$iNextEst
#         if (is.list(est)) est <- est[[1]]
#         if (!is.data.frame(est) || !"SC" %in% names(est)) return(NA_real_)
#         max(est$SC, na.rm = TRUE)
#     }
#     extract_asy <- function(res, diversity_label) {
#         ae <- res$AsyEst
#         if (!is.data.frame(ae)) return(c(NA_real_, NA_real_, NA_real_))
#         row <- ae[ rownames(ae) == diversity_label, c("Observed", "Estimator", "Est_s.e."), drop = FALSE]
#         print(row)
#         if (nrow(row) == 0) return(c(NA_real_, NA_real_, NA_real_))
#         as.numeric(row[1, ])
#     }

#     chain_len  <- sapply(lst, length)
#     reads_sum  <- sapply(lst, sum)
#     cpkr       <- ifelse(reads_sum > 0, (chain_len / reads_sum) * 1000, NA_real_)
#     avg_reads  <- sapply(lst, mean)
#     vmr        <- sapply(lst, function(x) { m <- mean(x); if (m > 0) stats::sd(x) / m else NA_real_ })
#     max_reads  <- sapply(lst, max)
#     singletons <- sapply(lst, function(x) sum(x == 1))
#     doubletons <- sapply(lst, function(x) sum(x == 2))

#     qD_max <- sapply(iNEXT_list, extract_qD_max)
#     SC_max <- sapply(iNEXT_list, extract_SC_max)

#     richness <- do.call(rbind, lapply(iNEXT_list, extract_asy, diversity_label = "Species Richness"))
#     shannon  <- do.call(rbind, lapply(iNEXT_list, extract_asy, diversity_label = "Shannon diversity"))
#     simpson  <- do.call(rbind, lapply(iNEXT_list, extract_asy, diversity_label = "Simpson diversity"))
# print(shannon)
#     div_stats <- matrix(ncol = 19, nrow = length(lst))
#     colnames(div_stats) <- c(
#         chain, "Reads", "CPKR", "Average_reads", "VMR", "Max_reads",
#         "Singletons", "Doubletons", "qD", "Sample_Coverage",
#         "observed_Richness", "estimated_Richness", "SE_Richeness",
#         "observed_Shannon", "estimated_Shannon", "SE_Shannon",
#         "observed_Simpson", "estimated_Simpson", "SE_Simpson"
#     )
#     rownames(div_stats) <- sample_names

#     div_stats[, chain] <- chain_len
#     div_stats[, "Reads"] <- reads_sum
#     div_stats[, "CPKR"] <- cpkr
#     div_stats[, "Average_reads"] <- avg_reads
#     div_stats[, "VMR"] <- vmr
#     div_stats[, "Max_reads"] <- max_reads
#     div_stats[, "Singletons"] <- singletons
#     div_stats[, "Doubletons"] <- doubletons
#     div_stats[, "qD"] <- qD_max
#     div_stats[, "Sample_Coverage"] <- SC_max
#     div_stats[, c("observed_Richness", "estimated_Richness", "SE_Richeness")] <- richness
#     div_stats[, c("observed_Shannon", "estimated_Shannon", "SE_Shannon")] <- shannon
#     div_stats[, c("observed_Simpson", "estimated_Simpson", "SE_Simpson")] <- simpson

#     out_file <- file.path(outpath, paste0("divstats_", chain, batchname, ".csv"))
#     utils::write.csv(div_stats, file = out_file, row.names = TRUE)

#     invisible(list(div_stats = div_stats, iNEXT_list = iNEXT_list))
# }

In [3]:
source("/Users/anabbi/git/ped_CapTCRseq/R/Immune_Diversity.R")

# Paths

In [4]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"

gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

# Main

In [5]:
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/data/mixcr/"

## TRB

In [5]:
file_list <- list.files(h4hpath, pattern = paste0("CLONES_", "TRB"), recursive = T)

In [ ]:
tail(file_list)

Some files are duplicated. Make a table to identify and pick the larger clone file

In [ ]:
file_df <- as.data.frame(file_list)

In [ ]:
file_df$samplename <- gsub(".*CLONES_TRB", "", file_df$file_list)
file_df$samplename <- gsub(".txt", "", file_df$samplename)

In [ ]:
file_df$size <- NA
for (i in 1:nrow(file_df)) {
    file_df$size[i] <- file.size(paste0(h4hpath, file_df$file_list[i]))
}


In [ ]:
head(file_df)

In [ ]:
file_df[ grepl("395-01", file_df$file_list),]

In [ ]:
#if duplicated keep the bigger file corresponding to deeper seqeuncing.
file_df_dedup <- file_df[0,]

for(s in unique(file_df$samplename)){
    tmp <- file_df[ file_df$samplename == s,]
    if(nrow(tmp) ==1 ){
        file_df_dedup <- rbind(file_df_dedup, tmp)
    }
    if(nrow(tmp) > 1 ){
        file_df_dedup <- rbind(file_df_dedup, tmp[ tmp$size == max(tmp$size),])
    }    
}

In [ ]:
file_df_dedup[ grepl("343", file_df_dedup$file_list),]

In [ ]:
file_df_dedup[ grepl("343-05", file_df_dedup$file_list),]

In [ ]:
dim(file_df)
dim(file_df_dedup)

In [ ]:
file_df_dedup_trb <- file_df_dedup
# save(file_df_dedup_trb, file = paste0(datapath,"capTCRseq/TRBfiles.RData"))

In [ ]:
batchTRB <- immunelistfx(file_df_dedup$file_list, h4hpath, "TRB")

In [ ]:
names(batchTRB) <- gsub(".*CLONES_TRB", "", names(batchTRB))

In [ ]:
head(names(batchTRB))

In [ ]:
TRBlist <- batchTRB 

In [ ]:
readr::write_rds(TRBlist, file = paste0(gitpath,"Data/TRBlist.rds"))

In [ ]:
save(TRBlist, file = paste0(datapath,"capTCRseq/TRBlist.RData"))

### divstats

In [6]:
load(file = paste0(datapath,"capTCRseq/TRBlist.RData"))
load(file = paste0(datapath,"capTCRseq/TRBfiles.RData"))

In [7]:
TRBlist <- TRBlist[sapply(TRBlist, function(x) length((x)) != 0)]
length(names(TRBlist))

[1] 584

In [8]:
# add filenames to divstats
myfilenames <- file_df_dedup_trb[ file_df_dedup_trb$samplename %in% names(TRBlist),]

In [9]:
head(myfilenames)

,file_list,samplename,size
,<chr>,<chr>,<dbl>
265,batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315-05-PBMC-DNA,16011
2,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-01-PBMC-DNA.txt,CHP_350-01-PBMC-DNA,85631
3,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-02-PBMC-DNA.txt,CHP_350-02-PBMC-DNA,143632
4,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-03-PBMC-DNA.txt,CHP_350-03-PBMC-DNA,107190
5,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-04-PBMC-DNA.txt,CHP_350-04-PBMC-DNA,54188
6,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_353-02-PBMC-DNA.txt,CHP_353-02-PBMC-DNA,9062


In [ ]:
save(TRBlist, file = "/Users/anabbi/Desktop/sam/intercept/TRBlist.RData")
save(file_df_dedup_trb, file = "/Users/anabbi/Desktop/sam/intercept/TRBfiles.RData")

In [ ]:
# parallel_inext.R ## run on valinor interactive 

### Cleanup divstats

In [10]:
divstats <- read.csv(paste0(datapath, "capTCRseq/divstats_TRBCHP_updated.csv"), header = T, stringsAsFactors = F, row.names = 1)

In [11]:
# add 9 samples with nonproductive cdr3
addsamples <- c(
    "CHP_347-01-cfDNA", "CHP_366-05_cfDNA", "CHP_377-01-cfDNA",
    "CHP_381-05_cfDNA", "CHP_391-02_cfDNA", "CHP_392-01_cfDNA",
    "CHP_398-01_cfDNA", "CHP_401-03_cfDNA", "CHP_419-FFPE-DNA_S2"
)


In [12]:
divstats[ nrow(divstats) + 9 , ] <- NA

In [13]:
rownames(divstats)[is.na(divstats$TRB)] <- addsamples

In [14]:
#example sample_id CHP_xxx_xx
divstats$sample_id <- rownames(divstats)

# remove trailing _cfDNA or PBMC-DNA from sample_id
divstats$sample_id <- gsub("_cfDNA$|_PBMC_DNA$|-cfDNA$|-PBMC_DNA$|-PBMC-DNA$|_PBMC-DNA", "", divstats$sample_id)


In [15]:
#example basename CHP_xxx-xx-PBMC-DNA
divstats$basename <- rownames(divstats)

In [16]:
# add filenames to divstats
divstats$file <- myfilenames$file_list[ match(divstats$basename, myfilenames$samplename)]

In [17]:
head(divstats[grepl("CHP_407", divstats$sample_id),])

,TRB,Reads,CPKR,Average_reads,VMR,Max_reads,Singletons,Doubletons,qD,Sample_Coverage,...,SE_Richeness,observed_Shannon,estimated_Shannon,SE_Shannon,observed_Simpson,estimated_Simpson,SE_Simpson,sample_id,basename,file
,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
CHP_407-01_PBMC_DNA,376,2065,182.0823245,5.492021,0.7235782,25,43,57,391.06694,0.9985318,...,10.5776306,296.625795,328.340127,4.82121497,247.015293,280.442163,6.77665004,CHP_407-01,CHP_407-01_PBMC_DNA,batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_407-01_PBMC_DNA.txt
CHP_407-02_PBMC_DNA,56,1003,55.8325025,17.910714,0.8493668,66,5,1,60.11956,0.9966608,...,9.6160107,40.144620,41.509305,0.91390958,32.776496,33.849983,1.02611973,CHP_407-02,CHP_407-02_PBMC_DNA,batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_407-02_PBMC_DNA.txt
CHP_407-03_cfDNA,4,352,11.3636364,88.000000,1.7322745,316,1,0,4.00000,1.0000000,...,0.5057732,1.481101,1.487815,0.05912977,1.230549,1.231358,0.03432500,CHP_407-03,CHP_407-03_cfDNA,batch11_cfDNA_11Sep2020/mixcr/clones/CLONES_TRBCHP_407-03_cfDNA.txt
CHP_407-03__PBMC-DN,33,1149,28.7206266,34.818182,0.9278100,157,2,0,33.86391,0.9997648,...,1.1807429,23.868713,24.224584,0.59816864,17.986145,18.256270,0.83346694,CHP_407-03__PBMC-DN,CHP_407-03__PBMC-DN,batch12_30Apr2021/mixcr/clones/CLONES_TRBCHP_407-03__PBMC-DN.txt
CHP_407-04-PBMC-DNA,569,69502,8.1868148,122.147627,0.7336426,659,9,4,574.96243,0.9999468,...,12.3872679,440.987356,442.840531,1.04361494,370.132939,372.109282,1.63606502,CHP_407-04,CHP_407-04-PBMC-DNA,batch13_26Aug2020/mixcr/clones/CLONES_TRBCHP_407-04-PBMC-DNA.txt
CHP_407-FFPE-DNA_S7,5,7275,0.6872852,1455.000000,0.9496434,3048,0,0,5.00000,1.0000000,...,0.0000000,3.243773,3.244667,0.02276269,2.904515,2.905275,0.02163161,CHP_407-FFPE-DNA_S7,CHP_407-FFPE-DNA_S7,batchFFPE_3Apr2020/mixcr/clones/CLONES_TRBCHP_407-FFPE-DNA_S7.txt


In [18]:
head(divstats[grepl("CHP_407", divstats$sample_id),])

,TRB,Reads,CPKR,Average_reads,VMR,Max_reads,Singletons,Doubletons,qD,Sample_Coverage,...,SE_Richeness,observed_Shannon,estimated_Shannon,SE_Shannon,observed_Simpson,estimated_Simpson,SE_Simpson,sample_id,basename,file
,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
CHP_407-01_PBMC_DNA,376,2065,182.0823245,5.492021,0.7235782,25,43,57,391.06694,0.9985318,...,10.5776306,296.625795,328.340127,4.82121497,247.015293,280.442163,6.77665004,CHP_407-01,CHP_407-01_PBMC_DNA,batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_407-01_PBMC_DNA.txt
CHP_407-02_PBMC_DNA,56,1003,55.8325025,17.910714,0.8493668,66,5,1,60.11956,0.9966608,...,9.6160107,40.144620,41.509305,0.91390958,32.776496,33.849983,1.02611973,CHP_407-02,CHP_407-02_PBMC_DNA,batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_407-02_PBMC_DNA.txt
CHP_407-03_cfDNA,4,352,11.3636364,88.000000,1.7322745,316,1,0,4.00000,1.0000000,...,0.5057732,1.481101,1.487815,0.05912977,1.230549,1.231358,0.03432500,CHP_407-03,CHP_407-03_cfDNA,batch11_cfDNA_11Sep2020/mixcr/clones/CLONES_TRBCHP_407-03_cfDNA.txt
CHP_407-03__PBMC-DN,33,1149,28.7206266,34.818182,0.9278100,157,2,0,33.86391,0.9997648,...,1.1807429,23.868713,24.224584,0.59816864,17.986145,18.256270,0.83346694,CHP_407-03__PBMC-DN,CHP_407-03__PBMC-DN,batch12_30Apr2021/mixcr/clones/CLONES_TRBCHP_407-03__PBMC-DN.txt
CHP_407-04-PBMC-DNA,569,69502,8.1868148,122.147627,0.7336426,659,9,4,574.96243,0.9999468,...,12.3872679,440.987356,442.840531,1.04361494,370.132939,372.109282,1.63606502,CHP_407-04,CHP_407-04-PBMC-DNA,batch13_26Aug2020/mixcr/clones/CLONES_TRBCHP_407-04-PBMC-DNA.txt
CHP_407-FFPE-DNA_S7,5,7275,0.6872852,1455.000000,0.9496434,3048,0,0,5.00000,1.0000000,...,0.0000000,3.243773,3.244667,0.02276269,2.904515,2.905275,0.02163161,CHP_407-FFPE-DNA_S7,CHP_407-FFPE-DNA_S7,batchFFPE_3Apr2020/mixcr/clones/CLONES_TRBCHP_407-FFPE-DNA_S7.txt


In [19]:
head(myfilenames[grepl("CHP_407", myfilenames$samplename),])

,file_list,samplename,size
,<chr>,<chr>,<dbl>
104,batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_407-01_PBMC_DNA.txt,CHP_407-01_PBMC_DNA,229053
105,batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_407-02_PBMC_DNA.txt,CHP_407-02_PBMC_DNA,34942
124,batch11_cfDNA_11Sep2020/mixcr/clones/CLONES_TRBCHP_407-03_cfDNA.txt,CHP_407-03_cfDNA,3563
139,batch12_30Apr2021/mixcr/clones/CLONES_TRBCHP_407-03__PBMC-DN.txt,CHP_407-03__PBMC-DN,21493
161,batch13_26Aug2020/mixcr/clones/CLONES_TRBCHP_407-04-PBMC-DNA.txt,CHP_407-04-PBMC-DNA,319668
407,batchFFPE_3Apr2020/mixcr/clones/CLONES_TRBCHP_407-FFPE-DNA_S7.txt,CHP_407-FFPE-DNA_S7,2682


In [20]:
# Fix sample_id for CHP_407-03__PBMC-DN (double underscore issue)
divstats$sample_id <- gsub("__PBMC-DN$", "", divstats$sample_id)
divstats$sample_id <- gsub("_-PBMC-DN$", "", divstats$sample_id)


In [21]:
divstats$sample_id <- gsub("[-_]$", "", divstats$sample_id)

Separate samples

In [22]:
#divstats$Sample <- NULL

In [23]:
divstats$Sample <- NA
divstats$Sample[grepl("PBMC", rownames(divstats))] <- "PBMC"
divstats$Sample[grepl("cfDNA", rownames(divstats))] <- "cfDNA"
divstats$Sample[grepl("FFPE", rownames(divstats))] <- "Tumor"

Separate LFS and non-LFS

In [24]:
divstats$Group <- NA
divstats$Group[grepl("LFS", rownames(divstats))] <- "LFS"
divstats$Group[grepl("CHP", rownames(divstats))] <- "CHP"

In [25]:
summary(divstats$Reads)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max.     NA's 
     1.0    841.8   3076.0  18632.0  12587.8 363060.0        9 

Get patient ids

In [26]:
divstats$Patient <- NA
# CHP
divstats$Patient[divstats$Group == "CHP"] <- substr(rownames(divstats)[divstats$Group == "CHP"],1,7)
# Normal samples have either YO or MO
divstats$Patient[grepl("YO", rownames(divstats))] <- gsub("_PBMC_DNA", "", rownames(divstats)[grepl("YO", rownames(divstats))])
divstats$Patient[grepl("MO", rownames(divstats))] <- gsub("_PBMC_DNA", "", rownames(divstats)[grepl("MO", rownames(divstats))])
# LFS
divstats$Patient[divstats$Group == "LFS"] <- substr(rownames(divstats)[divstats$Group == "LFS"],1,12)

Get cycles. For LFS, Normals and Tumors recode to 00

In [27]:
divstats$Cycle <- NA
divstats$Cycle[divstats$Group == "CHP"] <- substr(rownames(divstats)[divstats$Group == "CHP"],9,10)
divstats$Cycle[grepl("YO", rownames(divstats))] <- "00"
divstats$Cycle[grepl("MO", rownames(divstats))] <- "00"
divstats$Cycle[ divstats$Sample == "Tumor"] <- "00"
divstats$Cycle[ divstats$Group == "LFS"] <- "00"

In [28]:
divstats$sample_id <- NA

# bind patient and cycle for PBMC and cfdna to generate sample id
divstats$sample_id[divstats$Sample != "Tumor"] <- paste(divstats$Patient[divstats$Sample != "Tumor"], "_",
    divstats$Cycle[divstats$Sample != "Tumor"],
    sep = ""
)

# For tumor samples start with rownames because some of them are two biopsies from the same tumor
divstats$sample_id[divstats$Sample == "Tumor"] <- gsub("-FFPE-DN.*", "", rownames(divstats)[divstats$Sample == "Tumor"])


In [29]:
divstats$sample_id <- gsub("-", "_", divstats$sample_id)

In [30]:
divstats$Patient <- NULL
divstats$Cycle <- NULL
divstats$Sample <- NULL
divstats$Group <- NULL

In [31]:
write.csv(divstats,
    paste0(datapath, "capTCRseq/divstats_TRBCHP_updated_corrected.csv"),
    row.names = TRUE
)